In [1]:
from ydc.features.get_features import get_features
import pandas as pd
import numpy as np

from pprint import pprint

In [2]:
(features, df, box, combos, cells, neighbourhood) = get_features(status=True)

In [3]:
def _error(df_feat, quality, idx, coeffs):
    if idx is not None:
        return (quality[idx] - (df_feat[idx] * coeffs).sum(axis=1)).abs().mean()
    else:
        return (quality - (df_feat * coeffs).sum(axis=1)).abs().mean()
    
    
def get_coeffs_by_cat(df_feat, df_busi, combos):
    coeffs = {}
    errors = {}
    
    for combo in combos:
        idx = (df_busi['category'] == combo)
        
        c = get_coeffs(df_feat[idx], df_busi.loc[idx, 'stars'])
        coeffs[combo] = c
        
        errors[combo] = {
            "all": _error(df_feat, df_busi['stars'], None, c),
            "combo": _error(df_feat, df_busi['stars'], idx, c),
            "other": _error(df_feat, df_busi['stars'], -idx, c)
        }

    return coeffs, errors
    
def get_coeffs(df_feat, quality):
    if (df_feat.empty):
        return pd.Series(0, index=df_feat.columns)
    
    a = df_feat.values
    b = quality.values
    lstsq_res = np.linalg.lstsq(a,b)[0]
    
    return pd.Series(lstsq_res, index=df_feat.columns)

def find_best_location():
    return

In [5]:
coeffs, errors = get_coeffs_by_cat(features, df, combos)
print(pd.DataFrame(errors).transpose().mean())

c_all = get_coeffs(features,df['stars'])
e_all = _error(features, df['stars'], None, c_all)
print("---")
print(e_all)

all      11709065.322837
combo           0.547884
other    11724912.243139
dtype: float64
---
0.710769900863
